# **LLM Agents** with LangChain and OpenAI

In [ ]:
# Install the necessary packages
!pip install langchain -qU
!pip install langchain-openai -qU
!pip install langchain-community -qU

!pip install duckduckgo-search -qU
!pip install pyowm -qU

!pip install langchainhub -qU

In [ ]:
import os
from google.colab import userdata

### Initialize OpenAI LLM

In [ ]:
from langchain_openai import ChatOpenAI

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Initialize the ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

### Initialize DuckDuckGo Search Tool

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize DuckDuckGo search tool
search_tool = DuckDuckGoSearchRun()

search_tool

### Initialize OpenWeatherMap Tool

In [ ]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain.agents import load_tools

os.environ["OPENWEATHERMAP_API_KEY"] = userdata.get('OPENWEATHERMAP_API_KEY')

# Initialize OpenWeatherMap tool
weather = OpenWeatherMapAPIWrapper()

weather_tool = load_tools(["openweathermap-api"], llm)[0]

weather_tool

### Load Prompt Template from LangChainHub

In [ ]:
from langchain import hub

# Pull the prompt template from LangChainHub
prompt = hub.pull("hwchase17/react")

print(prompt)

### Create Built-in Agent

In [ ]:
from langchain.agents import create_react_agent

# List of tools for the agent
tools = [search_tool, weather_tool]

# Create the agent using the LLM and the prompt template
agent = create_react_agent(tools = tools, llm = llm, prompt=prompt)

In [ ]:
from langchain.agents import AgentExecutor

# Initialize the agent executor with the created agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

### Invoke the agent executor with an example question

In [ ]:
response = agent_executor.invoke({"input": "What's the weather in Kandy?"})

print(response['output'])

In [ ]:
response = agent_executor.invoke({"input": "what is machine learning?"})

print(response['output'])

In [ ]:
response = agent_executor.invoke({"input": "how many words count within introduction to machine learning?"})

print(response['output'])

### Define Custom Tool

In [ ]:
from langchain.agents import tool

@tool
def word_count(text: str) -> int:
  """Returns the word count."""
  return len(text.split())

In [ ]:
# Add the custom tool to the list of tools
tools.append(word_count)

print(tools)

In [ ]:
# Create a new agent with the DuckDuckGo search tool, OpenWeatherMap tool  and custom word count tool
agent = create_react_agent(tools = tools, llm = llm, prompt=prompt)

# Initialize the agent executor with the new agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

### Invoke the custom agent executor with example questions

In [ ]:
response = agent_executor.invoke({"input": "what is machine learning?"})

print(response['output'])

In [ ]:
response = agent_executor.invoke({"input": "how many words count within introduction to machine learning?"})

print(response['output'])

In [ ]:
response = agent_executor.invoke({"input": "what is the temperature of the city where 'lotus tower' is located?"})

print(response['output'])